XGBoost -- hyperparameter tuning and estmation of generalised error via CV

In [16]:
#import libraries
from sklearn.metrics import r2_score, make_scorer
from sklearn.feature_selection import SelectPercentile
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# from sklearn import preprocessing
from sklearn import neighbors
from sklearn import feature_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn import neural_network
from sklearn import svm
from sklearn import kernel_ridge
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import RFECV
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
import xgboost as xgb
from sklearn.cluster import DBSCAN
from sklearn.utils import resample
import warnings  
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE
from sklearn import svm
from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from bayes_opt import BayesianOptimization
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from utils import *

In [17]:
#define functions for loading data and producing final CSV 

'''
eliminate highly correlated features
'''
def to_be_eliminated(df):
    # Create correlation matrix
    corr_matrix = df.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    return to_drop

'''
loading training and test datasets
'''
def load_data():
    X_train = pd.read_csv("X_train.csv")
    X_test = pd.read_csv("X_test.csv")
    y_train = pd.read_csv("y_train.csv")
     
    #dropping id column
    X_train = X_train.drop('id', axis = 1)
    X_test = X_test.drop('id', axis = 1)
    y_train = y_train.drop('id', axis = 1)
   
    #reshuffling data 
    X_train['y'] = y_train
    X_train = X_train.sample(frac=1).reset_index(drop=True)
    y_train = X_train['y']
    X_train = X_train.drop('y', axis = 1)
    
    to_drop = to_be_eliminated(X_train)
    
    for i in range(len(to_drop)):
        X_train = X_train.drop(to_drop[i], axis = 1)
    
    for i in range(len(to_drop)):
        X_test = X_test.drop(to_drop[i], axis = 1)

    return X_train, X_test, y_train


'''
produce submission file
'''
def produce_solution(y):
    with open('out.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', lineterminator="\n")
        writer.writerow(['id', 'y'])
        for i in range(y.shape[0]):
            writer.writerow([float(i), y[i]])


In [ ]:
X_train_original, X_test_original, y_train_original = load_data()

X_train, X_val, y_train, y_val = train_test_split(X_train_original, y_train_original, test_size=0.2, random_state=4)

# Bayesian Optimization on XGB

In [ ]:
from bayes_opt import BayesianOptimization
from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold

# Specify the function
def xgbfun(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, objective='binary:logistic', 
           booster='gbtree', tree_method='auto', n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, 
           subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, 
           scale_pos_weight=1, base_score=0.5, random_state=42, missing=None, num_parallel_tree=1, 
           importance_type='gain'):
    
    xgbmodel = xgb.XGBClassifier(max_depth=int(max_depth), learning_rate=learning_rate, n_estimators=int(n_estimators), 
                              verbosity=1, objective=objective, booster=booster, tree_method=tree_method, n_jobs=1, 
                             gamma=gamma, min_child_weight=int(min_child_weight), max_delta_step=int(max_delta_step), 
                              subsample=subsample, 
                             colsample_bytree=colsample_bytree, colsample_bylevel=colsample_bylevel, 
                              colsample_bynode=colsample_bynode, 
                             reg_alpha=reg_alpha, reg_lambda=reg_lambda, scale_pos_weight=scale_pos_weight, 
                              base_score=0.5, random_state=random_state, 
                             missing=None, num_parallel_tree=1, importance_type='gain')
    
    xgb_smote_estimators = [
              ('standardization', preprocessing.StandardScaler()), 
              ('feature_selection', SelectFromModel(RandomForestClassifier(n_estimators=300, random_state=42))),
              ('classifier', SMOTEClassifier(SMOTE('minority'), xgb_model))
             ]
    xgb_smote_pipe = Pipeline(xgb_smote_estimators)
    
    
    scores = cross_val_score(estimator = xgb_smote_pipe,
                                 X = X_train,
                                 y = y_train,
                                 scoring = make_scorer(balanced_accuracy_score),
                                 cv = 2)
    return np.mean(scores)



# specify parameters and distributions to sample from
param_dist = {"max_depth": (2, 5),
              "learning_rate": (0.1, 0.9),
              "n_estimators": (1, 200),
              "gamma": (0.001, 2),
              "min_child_weight": (0.01, 3),
              "max_delta_step": (0.01, 3),
              "subsample": (0.01, 1),
              "colsample_bytree": (0.01, 1),
              "colsample_bylevel": (0.01, 1), 
              "colsample_bynode": (0.01, 1),
              "reg_alpha": (0, 1),
              "reg_lambda": (0, 1),
              "scale_pos_weight": (0.001, 1)}

optimizer = BayesianOptimization(
    f=xgbfun,
    pbounds=param_dist,
    verbose=2,
    random_state=1,
)

probe_params = { "max_depth":3, 
                "learning_rate":0.1, 
                "n_estimators":100, 
                "gamma":0, 
                "min_child_weight":1, 
                "max_delta_step":0, 
                "subsample":1, 
                "colsample_bytree":1, 
                "colsample_bylevel":1, 
                "colsample_bynode":1, 
                "reg_alpha":0, 
                "reg_lambda":1, 
                "scale_pos_weight":1 }
optimizer.probe(
    params=probe_params,
    lazy=True
)

optimizer.maximize(
    init_points=2,
    n_iter=200,
)

print(optimizer.max)

|   iter    |  target   | colsam... | colsam... | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.5563   |  1.0      |  1.0      |  1.0      |  0.0      |  0.1      |  0.0      |  3.0      |  1.0      |  100.0    |  0.0      |  1.0      |  1.0      |  1.0      |
|  2        |  0.5524   |  0.4229   |  0.7231   |  0.01011  |  0.6054   |  0.2174   |  0.2861   |  2.559    |  1.043    |  79.96    |  0.5388   |  0.4192   |  0.6855   |  0.2124   |
|  3        |  0.5578   |  0.8793   |  0.03711  |  0.6738   |  0.8352   |  0.547    |  0.4298   |  2.594    |  2.404    |  193.7    |  0.3134   |  0.6923   |  0.8765   |  0.8957   |
|  4        |  0.5443   |  0.06798  |  0.6587   |  0.3114   |  0.02448  |  0.1407   |  2.8